In [3]:
import pandas as pd
import numpy as np

In [17]:
data = pd.read_csv('diabetes.csv')

In [5]:
data.head()

,record_id,patient_id,measurement_date,measurement_27,measurement_28,measurement_37,measurement_42,measurement_43,measurement_68,measurement_76,measurement_89,measurement_65,measurement_119,measurement_67,measurement_107
0,0,69,2013-04-11,7.29,33.40,4.02,30.5,47.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,69,2013-04-30,7.16,24.57,4.08,39.7,54.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,69,2013-05-24,6.65,24.95,NaN,34.5,50.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,109,2013-02-06,6.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,109,2015-06-25,6.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df = data[data['measurement_27'].isna() == False]

In [30]:
df.shape

(33522, 15)

In [29]:
for column in df.drop(['record_id', 'patient_id', 'measurement_date'], axis=1):
    print(str(df[column].isna().sum()) + " " + column)

0 measurement_27
15531 measurement_28
12130 measurement_37
12665 measurement_42
12613 measurement_43
32770 measurement_68
32959 measurement_76
30520 measurement_89
31025 measurement_65
32779 measurement_119
33415 measurement_67
33425 measurement_107


In [45]:
df['measurement_28'].notna().values.sum()

17991

In [44]:
np.logical_and(df['measurement_28'].notna().values, df['measurement_27'].notna().values).sum()

17991

In [47]:
print(np.logical_and(df['measurement_28'].notna().values, df['measurement_42'].notna().values).sum())
print(np.logical_and(df['measurement_28'].notna().values, df['measurement_43'].notna().values).sum())
print(np.logical_and(df['measurement_42'].notna().values, df['measurement_43'].notna().values).sum())

16868
16871
20764


In [54]:
temp1 = np.logical_and(df['measurement_28'].notna().values, df['measurement_37'].notna().values)
temp2 = np.logical_and(df['measurement_42'].notna().values, df['measurement_43'].notna().values)
np.logical_and(temp1, temp2).sum()

12130

In [58]:
df2 = df.drop(['measurement_68','measurement_76','measurement_89','measurement_65',
               'measurement_119','measurement_67','measurement_107'], axis=1)
df2 = df2.dropna()

In [60]:
df2.head()

,record_id,patient_id,measurement_date,measurement_27,measurement_28,measurement_37,measurement_42,measurement_43
0,0,69,2013-04-11,7.29,33.40,4.02,30.5,47.6
1,1,69,2013-04-30,7.16,24.57,4.08,39.7,54.4
8,8,178,2016-03-11,4.70,12.87,5.05,34.3,72.4
9,9,217,2013-06-09,6.30,10.38,6.46,23.8,28.5
10,10,217,2014-12-13,5.82,15.26,5.81,32.4,37.4


In [87]:
df2.to_csv('four_features.csv', index=False)

In [61]:
msd = pd.read_csv('measurements_dict.csv')
msd.head()

,measurement_id,measurement_name
0,27,Глюкоза
1,28,Билирубин общий
2,37,Холестерин общий
3,42,Аспартатаминотрансфераза (АсАТ)
4,43,Аланинаминотрансфераза (АлАТ)


In [71]:
X = df2[['measurement_28', 'measurement_37', 'measurement_42', 'measurement_43']].values
y = df2['measurement_27'].apply(lambda x: 1 if x > 6.1 else 0).values
print(len(y))
print(y.sum())

12130
2755


In [75]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
import warnings
warnings.filterwarnings('ignore')
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
logit = LogisticRegression(n_jobs=-1, random_state=42)

In [85]:
X1 = df2[['measurement_28', 'measurement_37', 'measurement_42', 'measurement_43']].values
X2 = df2[['measurement_28', 'measurement_37', 'measurement_42']].values
X3 = df2[['measurement_28', 'measurement_37', 'measurement_43']].values
X4 = df2[['measurement_28', 'measurement_42', 'measurement_43']].values
X5 = df2[['measurement_37', 'measurement_42', 'measurement_43']].values

score1 = cross_val_score(estimator=logit, X=X1, y=y, cv=skf, scoring='roc_auc')
score2 = cross_val_score(estimator=logit, X=X2, y=y, cv=skf, scoring='roc_auc')
score3 = cross_val_score(estimator=logit, X=X3, y=y, cv=skf, scoring='roc_auc')
score4 = cross_val_score(estimator=logit, X=X4, y=y, cv=skf, scoring='roc_auc')
score5 = cross_val_score(estimator=logit, X=X5, y=y, cv=skf, scoring='roc_auc')

In [86]:
print(score1)
print(score2)
print(score3)
print(score4)
print(score5)

[0.60309867 0.58948845 0.62288453]
[0.57286198 0.56079791 0.59424662]
[0.59969166 0.58349054 0.62145987]
[0.63919896 0.629158   0.6290841 ]
[0.60191269 0.58902484 0.62168575]


In [92]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df2[['measurement_28', 'measurement_37', 'measurement_42', 'measurement_43']])
score = cross_val_score(estimator=logit, X=X_scaled, y=y, cv=skf, scoring='roc_auc')
score

array([0.60194159, 0.58834022, 0.62181194])

In [93]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df2[['measurement_37', 'measurement_43']])
score = cross_val_score(estimator=logit, X=X_scaled, y=y, cv=skf, scoring='roc_auc')
score

array([0.59775652, 0.58243747, 0.61971974])